In [34]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [42]:
flights_file = "Resources/flights.csv"
flights_df = pd.read_csv(flights_file, low_memory=False)
flights_df = flights_df.drop_duplicates(subset=['YEAR', 'MONTH', 'DAY', 'FLIGHT_NUMBER', 'ORIGIN_AIRPORT'],keep='first')
flights_df.head()


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
airlines_file = "Resources/airlines.csv"
airlines_df = pd.read_csv(airlines_file)
airlines_df.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [37]:
airports_file = "Resources/airports.csv"
airports_df = pd.read_csv(airports_file)
airports_df.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [38]:
weather_file = "Resources/warn-2015.xlsx"
weather_df = pd.read_excel(weather_file)
weather_df

,warn-2015,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,#This file contains experimental data.,NaN,NaN,NaN,NaN,NaN,NaN
1,#File written at Fri Jan 15 05:37:11 EST 2016.,NaN,NaN,NaN,NaN,NaN,NaN
2,#ISSUEDATE,EXPIREDATE,ISSUEWFO,MESSAGEID,MESSAGETYPE,WARNINGTYPE,POLYGON
3,2015-01-03 05:16:00.0,2015-01-03 09:15:00.0,KSHV,30516,WGUS54,FLASH FLOOD,POLYGON((-94.55 34.35 -95.26 33.38 -94.88 33.4...
4,2015-01-03 07:01:00.0,2015-01-03 10:00:00.0,KLZK,30701,WGUS54,FLASH FLOOD,POLYGON((-94.24 34.36 -94.22 34.35 -93.95 34.3...
...,...,...,...,...,...,...,...
29193,2015-12-30 22:40:00.0,2015-12-31 04:45:00.0,KFFC,302240,WGUS52,FLASH FLOOD,POLYGON((-84.91 31.93 -84.45 31.93 -84.18 32.1...
29194,2015-12-30 22:51:00.0,2015-12-31 03:00:00.0,KFFC,302251,WGUS52,FLASH FLOOD,POLYGON((-82.59 33.82 -82.57 33.87 -82.6 33.86...
29195,2015-12-30 23:13:00.0,2015-12-31 02:30:00.0,KFFC,302313,WGUS52,FLASH FLOOD,POLYGON((-84.94 33.22 -85.24 33.13 -85.19 32.8...
29196,2015-12-30 23:23:00.0,2015-12-31 02:45:00.0,KFFC,302323,WGUS52,FLASH FLOOD,POLYGON((-84.7 33.51 -84.94 33.23 -84.25 33.19...


In [39]:
weather_df.head()

,warn-2015,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,#This file contains experimental data.,NaN,NaN,NaN,NaN,NaN,NaN
1,#File written at Fri Jan 15 05:37:11 EST 2016.,NaN,NaN,NaN,NaN,NaN,NaN
2,#ISSUEDATE,EXPIREDATE,ISSUEWFO,MESSAGEID,MESSAGETYPE,WARNINGTYPE,POLYGON
3,2015-01-03 05:16:00.0,2015-01-03 09:15:00.0,KSHV,30516,WGUS54,FLASH FLOOD,POLYGON((-94.55 34.35 -95.26 33.38 -94.88 33.4...
4,2015-01-03 07:01:00.0,2015-01-03 10:00:00.0,KLZK,30701,WGUS54,FLASH FLOOD,POLYGON((-94.24 34.36 -94.22 34.35 -93.95 34.3...


In [44]:
flights_cols = ["MONTH", "DAY", "DAY_OF_WEEK", "AIRLINE", "FLIGHT_NUMBER", "ORIGIN_AIRPORT", "DESTINATION_AIRPORT", "DEPARTURE_DELAY", "ARRIVAL_DELAY", "DIVERTED", "CANCELLED", "CANCELLATION_REASON", "AIR_SYSTEM_DELAY", "SECURITY_DELAY", "AIRLINE_DELAY", "LATE_AIRCRAFT_DELAY", "WEATHER_DELAY"]
flights_transformed = flights_df[flights_cols].copy()
flights_out_file = 'Resources/clean_flights.csv'
flights_transformed.to_csv(flights_out_file, index = False, header=True)
print (flights_transformed)

         MONTH  DAY  DAY_OF_WEEK AIRLINE  FLIGHT_NUMBER ORIGIN_AIRPORT  \
0            1    1            4      AS             98            ANC   
1            1    1            4      AA           2336            LAX   
2            1    1            4      US            840            SFO   
3            1    1            4      AA            258            LAX   
4            1    1            4      AS            135            SEA   
...        ...  ...          ...     ...            ...            ...   
5819074     12   31            4      B6            688            LAX   
5819075     12   31            4      B6            745            JFK   
5819076     12   31            4      B6           1503            JFK   
5819077     12   31            4      B6            333            MCO   
5819078     12   31            4      B6            839            JFK   

        DESTINATION_AIRPORT  DEPARTURE_DELAY  ARRIVAL_DELAY  DIVERTED  \
0                       SEA           

In [41]:
weather_df.head()

,warn-2015,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,#This file contains experimental data.,NaN,NaN,NaN,NaN,NaN,NaN
1,#File written at Fri Jan 15 05:37:11 EST 2016.,NaN,NaN,NaN,NaN,NaN,NaN
2,#ISSUEDATE,EXPIREDATE,ISSUEWFO,MESSAGEID,MESSAGETYPE,WARNINGTYPE,POLYGON
3,2015-01-03 05:16:00.0,2015-01-03 09:15:00.0,KSHV,30516,WGUS54,FLASH FLOOD,POLYGON((-94.55 34.35 -95.26 33.38 -94.88 33.4...
4,2015-01-03 07:01:00.0,2015-01-03 10:00:00.0,KLZK,30701,WGUS54,FLASH FLOOD,POLYGON((-94.24 34.36 -94.22 34.35 -93.95 34.3...
